In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("responses (55).csv")
data = dataset.copy()

In [3]:
data.drop(columns=['Timestamp'], inplace=True)
data.drop(index=[21, 129], inplace=True)
data.loc[62, 'Sex'] = data.loc[136, 'Sex'] = 'Other'
data['Where do you live? (name of village/town/city)'] = data['Where do you live? (name of village/town/city)'].apply(lambda x: x.strip().lower())
data['What percentage of your total monthly income (or allowance) do you keep aside for active savings?'] = data['What percentage of your total monthly income (or allowance) do you keep aside for active savings?'].map(
    {
        'Do not have a stable monthly income': 'no income',
        '10% - 50%': '10% - 50%',
        '5% - 10%': '5% - 10%',
        'Do not have a monthly income': 'no income',
        '< 5%': '< 5%',
        '> 50%': '> 50%'
    }
)

In [4]:
column_rename = {
    'Where do you live? (name of village/town/city)': 'P1',
    'Do you live in a rural or urban area?': 'P2',
    'Sex': 'P3',
    'Age': 'P4',
    'Your medium of schooling': 'P5',
    'Your highest educational qualification': 'P6',
    'Your monthly household income (in INR)': 'P7',
    'Who do you live with in your residence (i.e., where you currently reside)? ': 'P8',
    "Your father's highest educational qualification": 'P9',
    "Your mother's highest educational qualification": 'P10',
    'Who is responsible for making day-to-day decisions about money in your household?': 'F1',
    'Do you prepare a budget to keep track of necessary expenses and savings?': 'F2',
    'How do you keep track of your monthly expenses?': 'F3',
    'In the past 12 months have you been personally saving money in any of the following ways?': 'F4',
    'What percentage of your total monthly income (or allowance) do you keep aside for active savings?': 'F5',
    'What do you usually do to grow the money that you manage to save each month?': 'F6',
    'If you lost your main source of income today, how long would you be able to continue your current living expenses without borrowing money or moving house?': 'F7',
    'In the last 12 months have you done the following?': 'S1',
    'For an online payment that has to be made by card, which one would you opt for generally?': 'A1',
    'You are looking to buy new headphones online and you have 2 choices, out of which you need to choose one. The first set of headphones are priced at ₹700, while the other set is marked at ₹1,000 with a 30% discount. Both the sets are similar in terms of quality, performance, and durability.': 'A2',
    'From the list given below, identify items that are classified as assets:': 'K1',
    "Suppose you buy a piece of land for  ₹5 lakhs. After 10 years, you sell this land for ₹7 lakhs. How much would your profit be after selling the land? (Assume you don't have to pay any kind of taxes)": 'K2',
    'You have borrowed ₹100 from a friend. Which would be the lower amount to pay back: ₹105 or ₹100 plus 3%?': 'K3',
    'Assume that the income tax rate for an annual income between ₹1 lakh and ₹2 lakhs is 5% and for an annual income between ₹2 lakhs and ₹4 lakhs, the income tax rate is 10%. If your annual income is ₹3 lakhs, your income tax would be:': 'K4',
    'You want to buy a phone worth ₹25,000 and you decide to save ₹5,000 each month. In your third month of savings period (which would be of 5 months), a relative gifts you ₹4,000 in form of cash. What would you most likely do with this money?' : 'K5',
    'Just when you were about to reach your target of saving ₹25,000 for your new phone, you hear about another new, different model worth ₹22,000 and see a huge number of people booking it, along with good recommendations from a lot of well known experts and analysts.': 'K6',
    "You have ₹200 to spend on lunch and you're hungry. You buy some food items worth ₹180 and you feel full again after eating. You are now left with ₹20 of your lunch budget. What are you most likely to do?": 'K7',
    'Which one of the following investments do you consider as safer: putting your entire money in one business/investment, or putting your money into multiple businesses/investments?': 'K8',
    "You lend Rs. 25 to a friend and they return you Rs. 25 the next day. How much interest did they pay on this loan?": 'K9',
    "Say someone puts Rs. 100 into a tax free, no fee savings account with guaranteed interest is 2% per year. What will be the amount in the account after 5 years?": 'K10',
    "Suppose 5 brothers are given Rs. 1000 and asked to divide among themselves. However they have to wait for a year to get the money. If inflation stays at X%, then in 1 year will they be able to buy": 'K11'
}
columns = [f"F{i}" for i in range(1,8)] + ["S1", "A1", "A2"] + [f"K{i}" for i in range(1,12)]
data = data.rename(columns=column_rename)[columns]

In [5]:
data['F1'] = data['F1'].map({
    'Someone else makes these decisions': 'passive',
    'You make these decisions by yourself': 'active',
    'You make these decisions with someone else': 'passive'
})
data['F2'] = data['F2'].map({
    'Yes, I do so each month': 'active',
    'Yes, but only sometimes': 'passive',
    'No, I do not keep a written record of my expenses and savings': 'passive'
})
data['F3'] = data['F3'].map({
    'I do not keep a written record of my expenses': 'no record',
    'I write things down on paper in the form of notes': 'notes',
    'I make use of spreadsheets or other similar tools': 'spreadsheet or same'
})
data['F4'] = data['F4'].apply(lambda x: x.replace("assets, NFTs", "assets,NFTs"))
data['S1'] = data['S1'].apply(lambda x: x.replace("(investments, loan, credits)", "(investments,loan,credits)"))

In [6]:
scored_df = pd.DataFrame()

In [7]:
scored_df['F1_old'] = data['F1']
scored_df['F2_old'] = data['F2']

def score_f1(data, row):
    if row['F1_old'] == 'active' and row['F2_old'] == 'active':
        return 1
    else:
        return 0     

In [8]:
scored_df['F1+F2'] = scored_df.apply(lambda row: score_f1(data=data, row=row), axis=1)
scored_df['F4'] = data['F4'].apply(lambda x: 1 if len(x.split(", "))>=2 else 0)
scored_df.drop(columns=['F1_old', 'F2_old'], inplace=True)
scored_df['F6'] = data['F6'].map({
    'I keep the money in my bank account': 1,
    'I invest this money periodically across various asset classes like bonds, stocks, cryptocurrency, fixed deposits, etc.': 1,
    "I'm generally not able to save money each month": 0,
    "I spend the saved money on shopping": 0
})


In [9]:
scored_df['K1'] = data['K1'].apply(lambda x: 1 if x == "Jewellery, Car, House, Art works" else 0)
scored_df['K2'] = data['K2'].map({
    'Rs. 2 lakhs': 0,
    'Not sure': 0,
    'Less than Rs. 2 lakhs': 1,
    'You will lose money': 1
})
scored_df['K3'] = data['K3'].map(lambda x: 1 if x=='Rs. 100 plus 3%' else 0)
scored_df['K4'] = data['K4'].map(lambda x: 1 if x=='Rs. 30,000' else 0)
scored_df['K5'] = data['K5'].map(lambda x: 1 if x=="Add it to your savings for your new phone and go on to save ₹1,000 more for that month" else 0)
scored_df['K7'] = data['K7'].map(lambda x: 1 if x == 'You keep the remaining money with yourself and do not buy anything else.' else 0)
scored_df['K8'] = data['K8'].map(lambda x: 1 if x == 'Multiple businesses/investment' else 0)
scored_df['K9'] = data['K9'].map(lambda x: 1 if x == 'No interest' else 0)
scored_df['K10'] = data['K10'].map(lambda x: 1 if x == 'More than Rs. 110' else 0)


In [10]:
scored_df

,F1+F2,F4,F6,K1,K2,K3,K4,K5,K7,K8,K9,K10
0,0,0,1,1,0,1,1,1,0,1,1,0
1,0,0,1,0,0,0,0,1,0,0,0,0
2,0,1,1,0,0,0,0,1,1,1,0,0
3,0,0,1,0,1,1,0,1,0,1,1,0
4,0,0,1,1,1,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
200,0,0,1,0,1,1,0,1,1,1,1,1
201,1,1,1,0,1,1,1,1,1,1,1,0
202,0,1,0,0,1,1,0,1,0,1,1,1
203,0,0,1,0,1,1,1,1,1,1,1,1


In [13]:
scored_df['TOTAL'] = scored_df.apply(np.sum, axis=1)

In [14]:
scored_df.to_csv('scores', index=False)